In [ ]:
!pip install astropy
!pip install googledrivedownloader

In [ ]:
from pathlib import Path
import os
from google_drive_downloader import GoogleDriveDownloader as gdd
import pickle
import zipfile
%load_ext autoreload 
%autoreload 2

In [ ]:
!ls ..

bin   datasets	home   lib64   mnt	  opt	run   storage  usr
boot  dev	lib    libx32  notebooks  proc	sbin  sys      var
cicc  etc	lib32  media   nvidia	  root	srv   tmp      workspace


In [ ]:
Path('../storage/dataset/').mkdir(parents=True, exist_ok=True)

In [ ]:
ls ../storage

Inverse/  dataset/  notebooks/  savepoints/


In [ ]:
Path('../storage/dataset/').mkdir(parents=True, exist_ok=True)
filepath = '../storage/dataset/parameters_base.fits'
if not os.path.isfile(filepath):
    print('Download from google disk')
    parameters_link = "https://drive.google.com/file/d/10HzzxlPoBnmqJb_v2NCyaUwO0ct6JD71/view?usp=sharing"
    fileid = '10HzzxlPoBnmqJb_v2NCyaUwO0ct6JD71'
    dest_path = '../storage/dataset/parameters_base.fits.zip'
    gdd.download_file_from_google_drive(file_id=fileid,
                                   dest_path=dest_path, showsize=True)
    with zipfile.ZipFile('../storage/dataset/parameters_base.fits.zip', 'r') as zip_ref:
        zip_ref.extractall('../storage/dataset/')
    os.remove('../storage/dataset/parameters_base.fits.zip')  

In [ ]:
fileid = '1oafzBpvf0ALFTVol6xzW6hW3RcLPXN2x'
dest_path = '../storage/dataset/20170905_030404.fits.zip'
gdd.download_file_from_google_drive(file_id=fileid,
                                dest_path=dest_path, showsize=True)
with zipfile.ZipFile('../storage/dataset/20170905_030404.fits.zip', 'r') as zip_ref:
    zip_ref.extractall('../storage/dataset/')
os.remove('../storage/dataset/20170905_030404.fits.zip') 

27.8 MiB        Done.


In [ ]:
ls ../storage/dataset

20170905_030404.fits   brand_b6000d100.tar.gz  prediction_ind.fits
__MACOSX/              gdrive.sh
brand_b100d100.tar.gz  parameters_base.fits


In [ ]:
!git clone https://github.com/iknyazeva/InverseProblem

Cloning into 'InverseProblem'...
remote: Enumerating objects: 592, done.
remote: Counting objects: 100% (592/592), done.
remote: Compressing objects: 100% (353/353), done.
remote: Total 592 (delta 428), reused 391 (delta 236), pack-reused 0
Receiving objects: 100% (592/592), 2.65 MiB | 1.58 MiB/s, done.
Resolving deltas: 100% (428/428), done.


In [ ]:
import sys
sys.path.append('InverseProblem')

In [ ]:
from inverse_problem.nn_inversion.main import HyperParams, Model
from inverse_problem.nn_inversion.transforms import normalize_output
from inverse_problem.nn_inversion.posthoc import compute_metrics, open_param_file, plot_params
from astropy.io import fits
import torch
import numpy as np
import os

In [ ]:
params = fits.open('../storage/dataset/parameters_base.fits')[0].data

In [ ]:
params[0]

array([1.20757286e+02, 8.65463943e+01, 1.21179359e+02, 2.79353294e+01,
       1.62066445e-01, 1.92586861e+01, 2.72371914e+04, 1.62790508e+04,
       6.21470988e-01, 4.35866445e-01, 2.30690539e-01])

In [ ]:
path_to_json =  "InverseProblem/res_experiments/hps_independent_mlp.json"
hps = HyperParams.from_file(path_to_json=path_to_json)
hps

HyperParams(hps_name='hps_independent_mlp', n_input=224, batch_norm=True, dropout=0.05, hidden_dims=[100, 100, 100], bottom_output=224, predict_ind=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], activation='elu', val_split=0.05, top_output=11, top_layers=2, transform_type='mlp_transform_rescale', mode='range', logB=True, factors=[1, 1000, 1000, 1000], cont_scale=40000, norm_output=True, source='database', hidden_size=100, bn=1, top_net='TopIndependentNet', bottom_net='ZeroMLP', lr=0.0001, lr_decay=0.005, weight_decay=0.0, batch_size=128, n_epochs=5, trainset=None, valset=None, patience=1, absolute_noise_levels=[109, 28, 28, 44])

In [ ]:
model = Model(hps)
#model.net

In [ ]:
Path('../storage/Inverse/trainlogs/').mkdir(parents=True, exist_ok=True)
Path('../storage/Inverse/trained_models/').mkdir(parents=True, exist_ok=True)

In [ ]:
!ls ../storage/Inverse

trained_models	trainlogs


In [ ]:
import shutil

In [ ]:
TRAIN = True
if TRAIN:
    history = model.train(data_arr=params,
                      logdir = "../storage/Inverse/trainlogs/", 
                      path_to_save='../storage/Inverse/trained_models/ind_mlp.pt')

batch:   0%|          | 0/24032 [00:00<?, ?it/s]


Epoch 001 train_loss: 0.1521          val_loss 0.0072



batch:   0%|          | 0/24032 [00:00<?, ?it/s]


Epoch 002 train_loss: 0.0055          val_loss 0.0053



epoch:  60%|██████    | 3/5 [3:46:02<2:32:26, 4573.26s/it]


Epoch 003 train_loss: 0.0052          val_loss 0.0051



batch:   0%|          | 0/24032 [00:00<?, ?it/s]


Epoch 004 train_loss: 0.0051          val_loss 0.0051



batch:  15%|█▌        | 3700/24032 [11:08<1:00:26,  5.61it/s]

In [ ]:
CTRAIN = True
if CTRAIN:
    model.continue_training('../storage/Inverse/trained_models/ind_mlp.pt', data_arr = params,
                      logdir = "/data/workspace_files/trainlogs/", 
                      path_to_save='/data/workspace_files/trained_models/ind_mlp.pt')

RuntimeError: Error(s) in loading state_dict for FullModel:
	Unexpected key(s) in state_dict: "top.task_layers.0.1.FC_layers.3.linear.weight", "top.task_layers.0.1.FC_layers.3.linear.bias", "top.task_layers.0.1.FC_layers.3.batch_normx.weight", "top.task_layers.0.1.FC_layers.3.batch_normx.bias", "top.task_layers.0.1.FC_layers.3.batch_normx.running_mean", "top.task_layers.0.1.FC_layers.3.batch_normx.running_var", "top.task_layers.0.1.FC_layers.3.batch_normx.num_batches_tracked", "top.task_layers.1.1.FC_layers.3.linear.weight", "top.task_layers.1.1.FC_layers.3.linear.bias", "top.task_layers.1.1.FC_layers.3.batch_normx.weight", "top.task_layers.1.1.FC_layers.3.batch_normx.bias", "top.task_layers.1.1.FC_layers.3.batch_normx.running_mean", "top.task_layers.1.1.FC_layers.3.batch_normx.running_var", "top.task_layers.1.1.FC_layers.3.batch_normx.num_batches_tracked", "top.task_layers.2.1.FC_layers.3.linear.weight", "top.task_layers.2.1.FC_layers.3.linear.bias", "top.task_layers.2.1.FC_layers.3.b

In [ ]:
!ls ../storage/Inverse/trained_models/

ind_mlp.pt  ind_mlp1.pt  ind_mlp_5ep.pt


In [ ]:
!cat ../storage/Inverse/trainlogs/history_hps_independent_mlp.txt

Train loss in epoch 0:  0.2949, val_loss: 0.0090
Train loss in epoch 1:  0.0068, val_loss: 0.0064
Train loss in epoch 2:  0.0063, val_loss: 0.0062
Train loss in epoch 3:  0.0060, val_loss: 0.0058
Train loss in epoch 4:  0.0058, val_loss: 0.0057


In [ ]:
shutil.copyfile('ind_mlp_5ep.pt', '../storage/Inverse/trained_models/ind_mlp_5ep.pt')

'../storage/Inverse/trained_models/ind_mlp_5ep.pt'

In [ ]:
from pathlib import Path
Path('InverseProblem/res_experiments/ind_mlp.pt').stat().st_size

6956221

In [ ]:
model.load_model('../storage/Inverse/trained_models/ind_mlp.pt')

In [ ]:
predicted, params = model.predict_full_image('../storage/dataset/20170905_030404.fits', '../storage/dataset/prediction_ind.fits')

In [ ]:
from inverse_problem.nn_inversion.posthoc import metrics

In [ ]:
param_score = []
for i in range(11):
    param_score.append(metrics(params[:, :, i], predicted[:, :, i]))


r2 0.42372166014440654
rmse 0.0911952627108014
mse 0.008519699465119986
mae 0.06686942715241348

r2 0.855972235076514
rmse 0.054889532469593216
mse 0.003127715710796275
mae 0.030736477790489745

r2 0.5587789320320499
rmse 0.17296730877987865
mse 0.030887749262352104
mae 0.09180101937208447

r2 0.6140851701066813
rmse 0.07980404849682887
mse 0.006412706393531415
mae 0.0657734480015369

r2 0.0873013793439008
rmse 0.13411552594006676
mse 0.01819365378582341
mae 0.10352890724162818

r2 -27.036770586411897
rmse 0.6880629931003045
mse 0.47992310614300615
mae 0.5794515163548253

r2 -9.34840380303171
rmse 0.20125868294576976
mse 0.04062056678320126
mae 0.19181903660457705

r2 -0.1619970456676841
rmse 0.10007970749489226
mse 0.010085058920222576
mae 0.0905935859585546

r2 0.830677212992509
rmse 0.018941086751164044
mse 0.00037845649714565065
mae 0.011180359168162991

r2 -0.49806253436553194
rmse 0.2935490009162807
mse 0.08723083406148262
mae 0.22942011330860237

r2 0.0647166513061676
rmse 0.078